train, val 데이터로 분리하는 코드

In [ ]:
# image 파일 리스트 생성 (폴더포함) 
import os
import random
import sys
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:

BASE_FOLDER = "/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet"

file_list = []
FOLDER_PREFIX = "image_motorcycle_helmet"
folders = os.listdir(BASE_FOLDER)
folders = [folder for folder in folders if folder != "classes.txt"]

# print(folders)
# exit()

for folder in folders:
    folder_path = os.path.join(BASE_FOLDER, folder)
    if not os.path.isdir(folder_path): continue

    for file in os.listdir(folder_path):
        if not file.endswith(".jpg"): continue

        txt_file = os.path.splitext(file)[0] + ".txt"
        # 해당하는 .txt 파일이 없으면 SKIP
        if not os.path.exists(os.path.join(folder_path, txt_file)): continue

        #line = f"{FOLDER_PREFIX}/{folder}/{file}"
        file_path = os.path.join(folder_path, file)
        output_path = os.path.join(folder_path, file_path)
        file_list.append(output_path)

print('total_num =', len(file_list), '개')

num = round(len(file_list) * 0.3)
print('val_num =', num, '개')
print('train_num =', len(file_list) - num, '개')

random.shuffle(file_list)
valid_list = file_list[:num]
train_list = file_list[num:]

with open('valid_data.txt','w',encoding='UTF-8') as f:
  for name in valid_list:
        f.write(name+'\n')

with open('train_data.txt','w',encoding='UTF-8') as f:
  for name in train_list:
        f.write(name+'\n')

total_num = 3012 개
val_num = 904 개
train_num = 2108 개


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

json -> txt로 분리하는 코드

In [ ]:
import os
import json
import cv2
import numpy as np
import random
import shutil

In [ ]:
CLASS_LABEL = "/content/gdrive/MyDrive/yolov4/train_motorcycle_helmet/motorcycle_helmet.names"

In [ ]:
f = open(CLASS_LABEL, 'r' , encoding='EUC-KR')
lines = f.readlines()

In [ ]:
class_list=[]
for i in range(4):
  class_list.append(lines[i].replace('\n',''))

In [ ]:
class_list

['helmet', 'head', 'cap', 'hood']

In [ ]:
all_class_dict = {}
for class_index, class_text in enumerate(class_list):
    all_class_dict[class_text] = class_index

print(all_class_dict)

total_class_dict = {}

def get_class_id_from_name(class_name):
    if class_name == "hat":
        class_name = "cap"

    class_id = all_class_dict[class_name]
    
    # class_id별 카운트 계산
    count = total_class_dict.get(class_id, 0)
    count += 1
    total_class_dict[class_id] = count

    return class_id


def get_yolo_line(class_text, box, image_width, image_height):
    
    class_text = class_text.strip()

    class_id = get_class_id_from_name(class_text)
    cx = float(box["x"] + box["w"]/2) /image_width     # 객체의 x 중심
    cy = float(box["y"] + box["h"]/2) /image_height    # 객체의 y 중심    
    rw = float(box["w"]) / image_width
    rh = float(box["h"]) / image_height
    yolo_line = f"{class_id} {cx:.6f} {cy:.6f} {rw:.6f} {rh:.6f}"
    return yolo_line


def convert_labelme_to_yolo(labelme_path, yolo_path):
    f = open(f'{labelme_path}')
    labelme_dict = json.load(f)
    yolo_lines = []
    for item in labelme_dict["shapes"]:
        class_text = item["label"]
        points = item["points"]

        # x1, y1, x2, y2 순서가 바뀌는 경우를 고려함
        x1 = points[0][0]
        y1 = points[0][1]
        x2 = points[1][0]
        y2 = points[1][1]
        if x1 > x2:
            x1 = points[1][0]
            x2 = points[0][0]
        if y1 > y2:
            y1 = points[1][1]
            y2 = points[0][1]

        box = {
            "x": x1,
            "y": y1,
            "w": x2 - x1,
            "h": y2 - y1,
        }    


        image_width = labelme_dict["imageWidth"]
        image_height = labelme_dict["imageHeight"]
    
        yolo_line = get_yolo_line(class_text, box, image_width, image_height)
        yolo_lines.append(yolo_line)

    yolo_label_txt = "\n".join(yolo_lines)
    with open(f'{yolo_path}','w',encoding='EUC-KR') as f:
      f.write(yolo_label_txt)

if __name__ == "__main__":

    # yolo txt 생성

    BASE_FOLDER = "/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet"
    # OUTPUT_FOLDER = r"E:\DATA\motorcycle_lp\yolo"


    # folders = ["ahnds"]
    # except_folders = ["경기"]
    folders = os.listdir(BASE_FOLDER)
    for folder in folders:
        print(folder)
        # if folder in except_folders:
        #     continue
            
        folder_path = os.path.join(BASE_FOLDER, folder)
        if not os.path.isdir(folder_path): continue

        for file in os.listdir(folder_path):
            print(file)
            file_path = os.path.join(folder_path, file)
            if file.endswith(".json"): 
                yolo_file = os.path.splitext(file)[0] + ".txt"
                output_path = os.path.join(folder_path, yolo_file)
                convert_labelme_to_yolo(file_path, output_path)

                print(output_path)

    print(total_class_dict)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
266_0.jpg
266_0.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/266_0.txt
267_0.jpg
267_0.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/267_0.txt
268_0.jpg
268_0.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/268_0.txt
268_1.jpg
268_1.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/268_1.txt
269_1.jpg
269_1.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/269_1.txt
269_2.jpg
269_2.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/269_2.txt
270_0.jpg
270_0.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/270_0.txt
270_1.jpg
270_1.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/270_1.txt
270_2.jpg
270_2.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/270_2.txt
271_0.jpg
271_0.json
/content/gdrive/MyDrive/yolov4/image_motorcycle_helmet/moonsw/271_0.txt
271_1.jpg
271_1.json
/content/gdri